In [4]:
import pandas as pd
df = pd.read_csv("almost_full_data4.csv")

In [71]:
df_tmp = df.sample(frac=0.1, random_state=42)

In [ ]:
bit_position = 6
df_tmp.dropna(subset=["Error_Status_Cloud"], inplace=True)
df_tmp["bit_6"] = df_tmp["Error_Status_Cloud"].apply(lambda x: (int(x) >> bit_position) & 1)
#df_tmp = df_tmp.drop(columns=["dataprofile_id", "cloud_received_time", "year", "year_month", "year_month_day","influx_migration_source_filename", "InstallationPosition_Write", "Override_Write","DN_Size", "Y3AnalogInputValue"])
df_tmp["error_in_next_30_days"] = 0
df_tmp["day_difference_from_first"] = 0  # Initialize with 0
df_tmp["day_difference_from_first"] = (df_tmp["sample_time"] - df_tmp["sample_time"].iloc[0]).dt.days

df_tmp = df_tmp.sort_values(by=["device_id", "sample_time"], ascending=[True, True])


In [73]:
from tqdm import tqdm

for i in tqdm(range(len(df_tmp) - 1, -1, -1), desc="Processing rows"):
    if df_tmp["bit_6"].iloc[i] == 1:
        time_sample = df_tmp["sample_time"].iloc[i]
        j = i
        curID = df_tmp["device_id"].iloc[i]

        while ((j > -1) and (df_tmp["device_id"].iloc[j] == curID) and 
               (df_tmp["day_difference_from_first"].iloc[j] > df_tmp["day_difference_from_first"].iloc[i] - 2)):
            df_tmp.at[j, "error_in_next_2_days"] = 1
            j -= 1

Processing rows:  11%|█         | 103695/969223 [01:58<16:27, 876.71it/s]   


KeyboardInterrupt: 